In [1]:
# Nativos
import random as rn
import os
import sys
import gc

#calculo
import numpy as np
import pandas as pd
import scipy as sp

#grafico
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
%matplotlib inline
sns.set(style="whitegrid")

#warning ignore future
import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
gc.collect()

BASE_DIR = os.path.dirname(os.getcwd())
if BASE_DIR not in sys.path: sys.path.append(BASE_DIR)

from utils import *
from graphs import *

SEED = 29082013
os.environ['PYTHONHASHSEED']=str(SEED)
np.random.seed(SEED)
rn.seed(SEED)

subfolder = "data"
os.listdir(subfolder)

/home/villacorta/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/home/villacorta/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


['CONTENT_CATEGORY.csv',
 '.ipynb_checkpoints',
 'pageviews_complemento.csv',
 'conversiones.csv',
 'sampleSubmission.csv',
 'pageviews.csv',
 'CONTENT_CATEGORY_TOP.csv',
 'pageview_with_previous.csv',
 'device_data.csv',
 'PAGE.csv',
 'SITE_ID.csv',
 'CONTENT_CATEGORY_BOTTOM.csv']

In [2]:
def get_schedule(val):
    if val < 6:
        return 'madrugada'
    elif val < 9:
        return 'antes_del_trabajo'
    elif val < 13:
        return 'trabajo_manana'
    elif val < 16:
        return 'almuerzo'
    elif val < 19:
        return 'trabajo_tarde'
    else:
        return 'luego_del_trabajo'
    
def get_day_cut(val):
    if val < 7:
        return 'inicio_mes'
    elif val < 13:
        return '7_12'
    elif val < 18:
        return 'quincena'
    elif val < 25:
        return '18_24'
    else:
        return 'fin_de_mes'
    
def get_trimestre(val):
    if val <= 3:
        return 1
    elif val <= 6:
        return 2
    elif val <= 9:
        return 3
    elif val <= 12:
        return 4
    return 0

In [3]:
col_user = 'USER_ID'
data = pd.read_csv(
    "{}/pageviews.csv".format(subfolder), parse_dates=["FEC_EVENT"], dtype=str
).sort_values(
    [col_user, "FEC_EVENT"]
)
data.shape

(17936934, 8)

In [4]:
comp = pd.read_csv(
    "{}/pageviews_complemento.csv".format(subfolder), parse_dates=["FEC_EVENT"], dtype=str
).sort_values(
    [col_user, "FEC_EVENT"]
)
comp.shape

(4933420, 8)

In [5]:
data.head()

,FEC_EVENT,PAGE,CONTENT_CATEGORY,CONTENT_CATEGORY_TOP,CONTENT_CATEGORY_BOTTOM,SITE_ID,ON_SITE_SEARCH_TERM,USER_ID
284,2018-01-01 09:56:47,14,4,2,4,2,1,0
285,2018-01-01 09:57:19,14,4,2,4,2,1,0
286,2018-01-01 09:57:48,12,4,2,4,2,1,0
287,2018-01-01 09:57:48,10,4,2,4,2,1,0
288,2018-01-01 09:57:49,11,4,2,4,2,1,0


In [6]:
comp.head()

,PAGE,CONTENT_CATEGORY,CONTENT_CATEGORY_TOP,CONTENT_CATEGORY_BOTTOM,SITE_ID,ON_SITE_SEARCH_TERM,USER_ID,FEC_EVENT
3136070,2,2,2,2,2,1,10000,2018-07-10 19:14:34
3136071,99,2,2,2,3,1,10000,2018-07-10 19:14:47
3136072,2,2,2,2,2,1,10000,2018-07-10 19:14:50
3136073,2,2,2,2,2,1,10000,2018-07-10 19:15:05
3136074,3,2,2,2,3,1,10000,2018-07-10 19:15:26


In [7]:
for col in data.columns:
    print(col, data[col].unique().shape)

print("/"*100)

for col in comp.columns:
    print(col, comp[col].unique().shape)

FEC_EVENT (10496454,)
PAGE (1725,)
CONTENT_CATEGORY (65,)
CONTENT_CATEGORY_TOP (13,)
CONTENT_CATEGORY_BOTTOM (65,)
SITE_ID (4,)
ON_SITE_SEARCH_TERM (294,)
USER_ID (11676,)
////////////////////////////////////////////////////////////////////////////////////////////////////
PAGE (1233,)
CONTENT_CATEGORY (59,)
CONTENT_CATEGORY_TOP (11,)
CONTENT_CATEGORY_BOTTOM (59,)
SITE_ID (4,)
ON_SITE_SEARCH_TERM (1,)
USER_ID (6661,)
FEC_EVENT (3209918,)


In [8]:
order_columns = list(data.columns)
order_columns

['FEC_EVENT',
 'PAGE',
 'CONTENT_CATEGORY',
 'CONTENT_CATEGORY_TOP',
 'CONTENT_CATEGORY_BOTTOM',
 'SITE_ID',
 'ON_SITE_SEARCH_TERM',
 'USER_ID']

In [9]:
data = pd.concat(
    [data[order_columns], comp[order_columns]], axis=0
)
del comp
data.shape, data.head()

((22870354, 8),
               FEC_EVENT PAGE CONTENT_CATEGORY CONTENT_CATEGORY_TOP  \
 284 2018-01-01 09:56:47   14                4                    2   
 285 2018-01-01 09:57:19   14                4                    2   
 286 2018-01-01 09:57:48   12                4                    2   
 287 2018-01-01 09:57:48   10                4                    2   
 288 2018-01-01 09:57:49   11                4                    2   
 
     CONTENT_CATEGORY_BOTTOM SITE_ID ON_SITE_SEARCH_TERM USER_ID  
 284                       4       2                   1       0  
 285                       4       2                   1       0  
 286                       4       2                   1       0  
 287                       4       2                   1       0  
 288                       4       2                   1       0  )

In [10]:
%%time
serie_count = data.groupby(by=[col_user])['FEC_EVENT'].count()
display(serie_count[serie_count <= 3])

USER_ID
11084    3
11317    3
11445    2
11538    3
11674    2
Name: FEC_EVENT, dtype: int64

CPU times: user 1.28 s, sys: 140 ms, total: 1.42 s
Wall time: 1.3 s


In [11]:
display(data.dtypes)

FEC_EVENT                  datetime64[ns]
PAGE                               object
CONTENT_CATEGORY                   object
CONTENT_CATEGORY_TOP               object
CONTENT_CATEGORY_BOTTOM            object
SITE_ID                            object
ON_SITE_SEARCH_TERM                object
USER_ID                            object
dtype: object

In [12]:
for col in data.columns:
    print(col, data[col].unique().shape)

FEC_EVENT (12397072,)
PAGE (1733,)
CONTENT_CATEGORY (65,)
CONTENT_CATEGORY_TOP (13,)
CONTENT_CATEGORY_BOTTOM (65,)
SITE_ID (4,)
ON_SITE_SEARCH_TERM (294,)
USER_ID (11676,)


In [13]:
new_data = []
for userx in data[col_user].unique():
    data_userx = data[data[col_user] == userx].reset_index(drop=True)
    shape_data = data_userx.shape[0]
    
    if shape_data <= 3:
        print("Only {} hiatorical:  ".format(shape_data), userx)
    
    
    if shape_data > 1:
        data_userx['PAGE_PREVIOUS'] = data_userx['PAGE'].shift(1)
        
        # data_userx['PAGE_PRE_PREVIOUS'] = data_userx['PAGE'].shift(2)
        data_userx = data_userx.drop([0],axis=0)

        data_userx['PAGE_PREVIOUS'] = data_userx[['PAGE', 'PAGE_PREVIOUS']].apply(
            lambda _: str(_.PAGE) + '_' + str(_.PAGE_PREVIOUS), axis=1
        )

        #data_userx['PAGE_PRE_PREVIOUS'] = data_userx[['PAGE_PREVIOUS', 'PAGE_PRE_PREVIOUS']].apply(
        #    lambda _: str(_.PAGE_PREVIOUS) + '_' + str(_.PAGE_PRE_PREVIOUS), axis=1
        #)

        new_data.append(data_userx)
    else:
        print("user with only one historical ::: ", userx)

data = pd.concat(new_data, axis=0)
del new_data
print(data.shape)

Only 3 hiatorical:   11084
Only 3 hiatorical:   11317
Only 2 hiatorical:   11445
Only 3 hiatorical:   11538
Only 2 hiatorical:   11674
(22858678, 9)


In [14]:
data.to_csv('data/pageview_with_previous', index=False)

In [ ]:
"""
Only 3 hiatorical:   11084
Only 3 hiatorical:   11317
Only 2 hiatorical:   11445
Only 3 hiatorical:   11538
Only 2 hiatorical:   11674

"""